In [17]:
import pandas as pd
import geopandas as gpd
from geopandas import GeoSeries
import folium
from shapely.geometry import Point, LineString, shape, mapping
import shapely.wkt
import pyproj
import os
import osmnx as ox
import ogr
import urllib.request
import geopy
from geopy.geocoders import Nominatim
import fiona
import shapefile
from pyproj import Proj, transform

In [18]:
if not os.path.exists('data'): os.makedirs('data')
if not os.path.exists('output'): os.makedirs('output')

Defining functions for getting the coordinates and city shapefile

In [19]:
# Gets the city boundary and saves it as a shapefile, if needed
def city_boundary(city):
    cityboundary = ox.gdf_from_place(city)  # Creates gdf from the place
    ox.save_gdf_shapefile(cityboundary, "city_boundary", "./data")  # Saves gdf as a shapefile

# Gets the coordinates from the city
def get_coordinates(name, city):
    geolocator = Nominatim(user_agent=name)
    location = geolocator.geocode(city)
    lat, lon = location.latitude, location.longitude
    print((lat, lon))
    SeeLocation = folium.Map(location=[lat, lon])
    folium.Marker(location=[lat, lon]).add_to(SeeLocation)
    return lat, lon, SeeLocation

Calling the coordinates function

In [21]:
city = 'Húsavík'
name = 'Gerard' # Write your name?
#city_boundary(city) # Call this if you want a shapefile of the city/municipality
lat, lon, AreaMap = get_coordinates(name, city)
#AreaMap # Call this if you want to check if the city/area is correct with a leaflet map

(66.0433168, -17.3410402)


Creating lines with equal latitude and longitude as the selected city

In [22]:
limit_lon = -179.9
limit_lon2 = 179.9

coordinates = {'LAT': [lat, lat], 'LON': [limit_lon, limit_lon2]}

same_lat = pd.DataFrame(data=coordinates)
same_lat = gpd.GeoDataFrame(same_lat, geometry=gpd.points_from_xy(same_lat.LON, same_lat.LAT))
same_lat.crs = {'init': 'epsg:4326'} #epsg4326

#gjson = same_lat.to_json()
geom0_lat = same_lat.loc[0]['geometry']
geom1_lat = same_lat.loc[1]['geometry']

# Create LineString from coordinates
startlat, endlat = [(geom0_lat.x, geom0_lat.y), (geom1_lat.x, geom1_lat.y)]
line_lat = LineString([startlat, endlat])

In [23]:
limit_lat = -89.9
limit_lat2 = 89.9

coordinates = {'LAT': [limit_lat, limit_lat2], 'LON': [lon, lon]}

same_lon = pd.DataFrame(data=coordinates)
same_lon = gpd.GeoDataFrame(same_lon, geometry=gpd.points_from_xy(same_lon.LON, same_lon.LAT))
same_lon.crs = {'init': 'epsg:4326'}

#gjson = same_lat.to_json()
geom0_lon = same_lon.loc[0]['geometry']
geom1_lon = same_lon.loc[1]['geometry']

# Create LineString from coordinates
startlon, endlon = [(geom0_lon.x, geom0_lon.y), (geom1_lon.x, geom1_lon.y)]
line_lon = LineString([startlon, endlon])
print(line_lon)

LINESTRING (-17.3410402 -89.90000000000001, -17.3410402 89.90000000000001)


In [24]:
line_lon, line_lat
gs_linelon = gpd.GeoSeries([line_lon])
gs_linelat = gpd.GeoSeries([line_lat])

gslonbuffer = gs_linelon.buffer(0.03)
gslonbuffer.crs = {'init': 'epsg:4326'}

gslatbuffer = gs_linelat.buffer(0.03)
gslatbuffer.crs = {'init': 'epsg:4326'}

Plotting the lines in a leaflet map

In [25]:
mapa = folium.Map(location=[lat, lon])
#points = folium.features.GeoJson(gjson)
#mapa.add_children(points)
#folium.GeoJson(line_lat).add_to(mapa)
#folium.GeoJson(line_lon).add_to(mapa)
folium.GeoJson(gslonbuffer).add_to(mapa)
folium.GeoJson(gslatbuffer).add_to(mapa)

mapa

In [14]:
mapa.save("LongLatLleida.html")

In [115]:
pd.set_option('display.max_colwidth', -1)
print(list(line_lat.coords))
print(gslatbuffer)

[(-179.9, 41.6147605), (179.9, 41.6147605)]
0    POLYGON ((179.90000 41.66476, 179.90490 41.66452, 179.90975 41.66380, 179.91451 41.66261, 179.91913 41.66095, 179.92357 41.65886, 179.92778 41.65633, 179.93172 41.65341, 179.93536 41.65012, 179.93865 41.64648, 179.94157 41.64254, 179.94410 41.63833, 179.94619 41.63389, 179.94785 41.62927, 179.94904 41.62452, 179.94976 41.61966, 179.95000 41.61476, 179.94976 41.60986, 179.94904 41.60501, 179.94785 41.60025, 179.94619 41.59563, 179.94410 41.59119, 179.94157 41.58698, 179.93865 41.58304, 179.93536 41.57941, 179.93172 41.57611, 179.92778 41.57319, 179.92357 41.57066, 179.91913 41.56857, 179.91451 41.56691, 179.90975 41.56572, 179.90490 41.56500, 179.90000 41.56476, -179.90000 41.56476, -179.90490 41.56500, -179.90975 41.56572, -179.91451 41.56691, -179.91913 41.56857, -179.92357 41.57066, -179.92778 41.57319, -179.93172 41.57611, -179.93536 41.57941, -179.93865 41.58304, -179.94157 41.58698, -179.94410 41.59119, -179.94619 41.59563, -179.947

C:\Users\Gerard\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


In [105]:
inProj = Proj(init='epsg:4326')
outProj = Proj(init='epsg:3857')
x1, y1 = lon, lat
x2, y2 = transform(inProj, outProj, x1, y1)

C:\Users\Gerard\Anaconda3\lib\site-packages\pyproj\crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))
C:\Users\Gerard\Anaconda3\lib\site-packages\pyproj\crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))


TypeError: transform() missing 1 required positional argument: 'y'